In [26]:
from models import *

In [27]:

device = 'cuda' if torch.cuda.is_available else 'cpu'
if device != 'cuda': print("No cuda detected. Consider colab.research.google.com for free GPUs.")

agent = ImpalaCNN().to(device)

In [28]:
print(agent)

ImpalaCNN(
  (block1): ImpalaBlock(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (res1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (res2): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): ImpalaBlock(
    (conv1): Conv2d(16, 32, ker

In [29]:
from grad_cam_batch import GradCam
import copy

In [30]:
class simple_a2cppo_gradcam:
    
    feature_modules = ["block1", "block2", "block3"]
    target_layer_names = ["conv1", "res1", "res2"]
    
    def __init__(self, model, feature_module_str="block2", target_layer_str="res2", use_cuda=True):
        cam_model = copy.deepcopy(model)
        
        self.cam_model = cam_model
        del cam_model.critic
        
        self.action_size = cam_model.actor.out_features
        
        self.model = model
        
        assert feature_module_str in self.feature_modules
        assert target_layer_str in self.target_layer_names
        
        self.grad_cam = GradCam(model=cam_model,
                        feature_module=getattr(cam_model, feature_module_str),
                        target_layer_names=[target_layer_str],
                        use_cuda=use_cuda)

In [31]:
a = simple_a2cppo_gradcam(agent, feature_module_str="block3", target_layer_str="conv1")